# Demo Notebook

# Photometry example using DS9 and Photutils

This notebook shows an example of some quick photometry investigation using imexam the photutils package.

**In order to run you must install photutils, an astropy affiliated photometry package.**

You can install the latest stable version using: " pip install --no-deps photutils "

In [ ]:
#The nbAgg backend allows for nice plotting control inside the notebook,qt5agg should also work  
#%matplotlib notebook

In [ ]:
# Import the required modules.
import os
from astropy.io import ascii, fits
import matplotlib.pyplot as plt
import matplotlib.image as mpimage
import numpy as np
import photutils
from photutils.detection import DAOStarFinder

## Read in Data
you can choose an example image of your own, if so replace the image name below

In [ ]:
# Load the image we are going to use in read-only mode
# 
hdulist = fits.open('iacs01t4q_flt.fits')

In [ ]:
# Similar to catfits in iraf
hdulist.info()

In [ ]:
print(f'Instrument: {hdulist[0].header["instrume"]}')
print(f'Detector: {hdulist[0].header["detector"]}')
print(f'Target Name: {hdulist[0].header["targname"]}')


## Let's Science!

We're going to show you two different ways to do this. 
The first example is just a quick way to look at your image

In [ ]:
#display a quick image to see what we are working with, you can even use matplotlib here
image = hdulist[1].data
plt.imshow(image, vmin=-1.5, vmax=1.5, cmap=plt.cm.gray, origin='lower')
plt.colorbar() #for reference       

In [ ]:
#how about displaying our image in DS9 so we can interact with it a bit more?
#Here I'll import imexam and open a connection to ds9 on my desktop
import imexam
a=imexam.connect() #let ds9 start before starting the next cell

In [ ]:
a.load_fits('iacs01t4q_flt.fits')
a.scale()
a.zoomtofit()

In [ ]:
# Extract keyword values that we will need.
# Note: Keyword names and units may vary for other instruments. Consult DHB.

header = hdulist[0].header
exptime   = header['EXPTIME']   # seconds
photflam  = header["PHOTFLAM"]  # ergs cm^-2 ang^-1 s^-1
photplam  = header["PHOTPLAM"]  # erg cm^-2 s^-1 Hz^-1


In [ ]:
stzpt = -2.5 * np.log10(photflam* 1.0) - 21.1
abmag = -2.5 * np.log10(photflam*1.0) - 21.1 -  5 * np.log10(photplam) + 18.692

In [ ]:
print('Exposure time: {0}'.format(exptime))
print('STMAG zeropoint: {0}'.format(stzpt))
print('ABMAG zeropoint: {0}'.format(abmag))

## Finding Sources - using photutils

In [ ]:
image = hdulist[1].data  # Input will be EXT 1, same as above

In [ ]:
#now we subtract a median background from the image, consider what you are doing here,'
#taking the straight median might not be the best way in such a crowded field. We will
#Use the background subtracted image for the rest of the example
skybkg=np.median(image)
print(skybkg)
image -= skybkg

In [ ]:
#let's also get the background deviation
from astropy.stats import median_absolute_deviation as mad
bkg_sigma = 1.4826 * mad(image)
print(bkg_sigma)

In [ ]:
#now we'll use the daofind method in photutils to find out sources
sources = DAOStarFinder(fwhm=2.5, threshold=3.*bkg_sigma)

In [ ]:
# See just a few lines of output, with no print it uses a pretty print in the notebook
source_cat=sources.find_stars(image)

In [ ]:
source_cat

## Visualizing the Sources
<p align="left">... and taking advantage of astropy tables </p>

In [ ]:
# what does daofind return?
print(type(sources))

In [ ]:
# extract xcenter values
xcen=source_cat['xcentroid']
print(xcen)

In [ ]:
# Plot of the sharpness versus the mag as blue pluses
plt.plot(source_cat['sharpness'], source_cat['mag'], 'b+')
plt.title('Mag vs. Sharpness from photutils')
plt.ylabel('Mag')
plt.xlabel('Sharpness')

# Save the plot as a PDF.
# Matplotlib automatically determines format from given extension.
#plt.savefig('sharp_v_mag_photutil.pdf')

In [ ]:
# Select stars with sharpness greater than 0.9
# use boolean arrays

#Want to save a table of just the sharper sources? You can do this:
sharp_sources = source_cat[source_cat['sharpness'] >0.95]
sharp_sources

print('Number of sharp stars: {0}'.format(len(sharp_sources)))

In [ ]:
# Plot the image again in a different color using matplotlib, just as an example
plt.imshow(image, vmin=-3., vmax=3., cmap=plt.cm.jet)

# Only show the high sharpness stars as black circles.
plt.plot(sharp_sources['xcentroid'], sharp_sources['ycentroid'], 'ko', mfc='None')

In [ ]:
#now lets do the same thing in DS9
stars = zip(sharp_sources['xcentroid'],sharp_sources['ycentroid'],sharp_sources['id'])
stars

In [ ]:
a.mark_region_from_array(stars)

## Aperture Photometry

In [ ]:
#photutils takes an aperture specfication along with the image as input:
#You can do a local background subtraction by specifying the apertures for the background in a similar manner
from photutils import aperture_photometry, CircularAperture
positions = [(x,y) for x,y in zip(sharp_sources['xcentroid'], sharp_sources['ycentroid'])]   
apertures = CircularAperture(positions, r=4.)    
phot_table = aperture_photometry(image, apertures)    
phot_table

## Analyze Photometry Results

Let's make a histogram of the recovered instrumental magnitudes

In [ ]:
mags=phot_table['aperture_sum']

In [ ]:
#wfc3 ir data is already in countrate, but no aperture correction has been applied here yet
goodmags=-2.5 *np.log10(mags[mags>0.] * photflam) - 21.1 

In [ ]:
#Nifty, right? Now lets plot a histogram of our values
plt.hist(goodmags, bins=20)
plt.xlabel('Mag')
plt.ylabel('N')
plt.title('Recovered Valid Magnitudes')

Note: The mags above are different from the mags in the iraf example mostly because a different zeropoint was used (as input to the phot task in the pars parameter set)

## Closing Time

In [ ]:
# It is a good practice to close any open file pointers.
# You might also want to check out the notes on  using the "with open as" clause
hdulist.close()

In [ ]:
a.close() #close the ds9 window